In [ ]:
def split_dataset(data, per_train, per_test):
    df = data
    train = df[0:int(np.ceil(df.shape[0]*per_train))]
    val = df[int(np.ceil(df.shape[0]*per_train)): int(np.ceil(df.shape[0]*(per_train+per_test)))]
    test = df[int(np.ceil(df.shape[0]*(per_train+per_test))):]
    #Escalado en train y test por separado. Si no se hace aquí ya no se puede porque todo lo que sigue es con arrays 3D
    scaler_y = StandardScaler()
    train_y = scaler_y.fit_transform(train[:,0].reshape(-1, 1))
    val_y = scaler_y.fit_transform(val[:,0].reshape(-1, 1))
    test_y = scaler_y.transform(test[:,0].reshape(-1, 1))
    scaler_x = StandardScaler()
    train_x = scaler_x.fit_transform(train[:,1:]) 
    val_x = scaler_x.fit_transform(val[:,1:]) 
    test_x = scaler_x.transform(test[:,1:])
    
    #Rehacemos los DataFrames con los datos escalados para poder hacer el split
    train = pd.concat([pd.DataFrame(train_y),pd.DataFrame(train_x)], axis=1)
    test = pd.concat([pd.DataFrame(test_y),pd.DataFrame(test_x)], axis=1)
    val = pd.concat([pd.DataFrame(val_y),pd.DataFrame(val_x)], axis=1)

    train = array(train)
    test = array(test)
    val = array(val)
    return train, test, val, scaler_x, scaler_y

In [ ]:
def to_supervised(train, n_input, n_out, steps_pred):
    # Ponemos los datos en 2D: una entrada para cada muestra y paso de tiempo con las n características
    data = train#.reshape((train.shape[0]*train.shape[1], train.shape[2])) 
    X, y = list(), list()
    in_start = 0
    # Pasamos por todo el conjunto de datos, de step en step
    for _ in range(len(data)):
        # Definimos cuando empieza el input, cuando acaba, e igual para el output
        # En este caso el output empieza cuando acaba el input
        in_end = in_start + n_input
        out_end = in_end + n_out
        # Iteramos mientras haya ejemplos
        if out_end <= len(data):
            X.append(data[in_start:in_end, :]) # X son todas las variables en t
            y.append(data[in_end:out_end, 0])  # y es la temperatura en t+1 (si n_out=1) #AQUÍ SE DEFINE EL LAG
        # Pasamos al siguiente instante de tiempo
        in_start += steps_pred
    X = array(X)
    y = array(y)
    return X, y

In [ ]:
def predict(datos_test_x, datos_test_y, n_input, modelo, steps, features):
    test_y = datos_test_y
    test_x = datos_test_x
    n_timesteps=steps
    n_features=features
    #Creamos una lista vacía para las predicciones
    predictions = list()
    #Ponemos el índice i a 0
    i=0
    for i in range(len(test_x)):
        data = test_x[i,:,:]
        input_x = data.reshape((1, n_timesteps, n_features))
        
        # Prediccion para cada paso de tiempo
        yhat = model.predict(input_x, verbose=0)
        yhat_sequence = yhat[0] 

        # Guardamos las predicciones
        predictions.append(yhat_sequence)
        
        if i < len(test_x):
            try: test_x[i+1,n_input,0] = yhat[0]
            except IndexError: continue
        else: break
            
    predictions = array(predictions)
    
    return predictions, test_y

In [ ]:
def metrics(test_y, prediction):
    rmse = np.sqrt(mean_squared_error(test_y,prediction))
    cvrmse = rmse/np.mean(test_y)*100
    r2 = r2_score(test_y, prediction)
    return cvrmse, r2, rmse